In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from subprocess import check_output

In [7]:
laliga=pd.read_csv('FMEL_Dataset.csv')
laliga=laliga[laliga.division==1]

In [8]:
laliga.head()

,id,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date,timestamp
0,1,1970-71,1,1,Atletico de Bilbao,Barcelona,1,1,12/09/1970,21938400
1,2,1970-71,1,1,Las Palmas,Atletico de Madrid,1,1,12/09/1970,21938400
2,3,1970-71,1,1,Real Madrid,Valencia,2,0,12/09/1970,21938400
3,4,1970-71,1,1,Celta de Vigo,Sporting de Gijon,2,0,13/09/1970,22024800
4,5,1970-71,1,1,Elche,Granada,1,1,13/09/1970,22024800


In [9]:
laliga.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17169 entries, 0 to 36684
Data columns (total 10 columns):
id              17169 non-null int64
season          17169 non-null object
division        17169 non-null int64
round           17169 non-null int64
localTeam       17169 non-null object
visitorTeam     17169 non-null object
localGoals      17169 non-null int64
visitorGoals    17169 non-null int64
date            17169 non-null object
timestamp       17169 non-null int64
dtypes: int64(6), object(4)
memory usage: 1.4+ MB


In [10]:
laliga['local_team_won']=laliga.apply(lambda row: 1 if row['localGoals']>row['visitorGoals'] else 0,axis=1)
laliga['visitor_team_won']=laliga.apply(lambda row: 1 if row['localGoals']<row['visitorGoals'] else 0,axis=1)
laliga['draw']=laliga.apply(lambda row: 1 if row['localGoals']==row['visitorGoals'] else 0,axis=1)